Cereals-and-bakery-products  Meats,-poultry,-fish,-and-eggs  \
0                         79.7                            90.4   
1                         80.6                            89.7   
2                         81.2                            90.3   
3                         82.4                            89.3   
4                         83.2                            88.0   
5                         83.7                            87.8   
6                         84.4                            89.9   
7                         84.8                            93.2   
8                         85.2                            95.7   
9                         86.4                            96.0   

   Dairy-and-related-products  Fruits-and-vegetables  Other-food-at-home  \
0                        87.3                   76.5                83.0   
1                        87.8                   76.0                84.3   
2                        88.1                

In [4]:
## Ran on python 3.9.18
from splinter import Browser # 0.19.0
from bs4 import BeautifulSoup # 4.11.1
import numpy as np # 1.26.0
import pandas as pd # 2.1.1
import matplotlib.pyplot as plt # 3.8.0
## These last imports are important to find appropriate driver to run on chrome version
# the version of selenium and webdriver may be the most important, these version allow for searching for right driver
from selenium import webdriver # 4.11.2
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # 4.0.1
from datetime import datetime 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Grab row by ID of category and pull value from that year
def parseRow(page,id):
    row= page.find('tr', attrs={'data-tt-id':id})
    cat = row.find('span',{'class':"fred-rls-elm-nm"})['id'].strip('}')
    val = row.find('td',{'headers':"table-unit-subheading1"}).get_text().split('Index')[0].strip()
    return val, cat

# Dictionary of categories and associated id
catId = {"Cereals-and-bakery-products": "34566",
         "Meats,-poultry,-fish,-and-eggs": "34567",
         "Dairy-and-related-products":"34568",
         "Fruits-and-vegetables":"34569",
         "Other-food-at-home":"34572"}

# start date and end date
endYear = 2023
startYear = 1980
month = 10
day = 1

## Iterate through years, relaoding page each time there is a new year
foodDict = {}
dateLst = []
for year in np.arange(startYear,endYear+1,1):
    for month in np.arange(1,13,1):
        date = datetime.strptime(f'{month:02d}/{day:02d}/{year}','%m/%d/%Y')
        dateLst.append(date)
        landing = f'https://fred.stlouisfed.org/release/tables?rid=10&eid=34561&od={year}-{month:02d}-{day:02d}#'
        driver.get(landing)
        html = driver.page_source
        mainPage = BeautifulSoup(html, 'html.parser')
        for cat in catId:
            catVal,foundCat = parseRow(mainPage,catId[cat])
            ## Check that the category ID still matches
            ## To do: implement code to account for changing id (not encountered yet)
            if cat != foundCat:
                print('Doesn\'t work')
            else:
                if foundCat not in foodDict:
                    foodDict[foundCat] = []
                foodDict[foundCat].append(float(catVal))

## Create test dataframe, this is just a sample of what we can do with it
test = pd.DataFrame(foodDict)
test['Date'] = dateLst
for cat in catId:
    plt.plot(test['Date'],test[cat],label = cat)
plt.legend()
plt.grid()
plt.show()

print(test.head(10))

driver.close()